# This details the process of setting started with Google compute engine (GCE).  This is more or less everything from scratch.

The CCCB account was already setup and tied to billing.  My brianlawney@gmail.com account is associated with that project since I was invited by Yaoyu.

I want to eventually use the python API.  Hence, I started from this page:
https://cloud.google.com/compute/docs/tutorials/python-guide#setup

From that tutorial, I followed the instructions at https://cloud.google.com/sdk/#nix to get the SDK for linux/mac.  This gets the command line tools gcloud, gcutil, etc.  When working with GCE services, you can use those directly, or use them indirectly via the Python hooks.

I downloaded the SDK tarball (https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz), unpacked, and ran the install script:

    brian@brian-XPS-13-9343 ~ $ wget https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
brian@brian-XPS-13-9343 ~ $ tar -xzf google-cloud-sdk.tar.gz
    brian@brian-XPS-13-9343 ~ $ cd google-cloud-sdk/
brian@brian-XPS-13-9343 ~/google-cloud-sdk $ ./install.sh


The install script asks some questions:
- Answered 'n' to improve sdk
- Answered 'y' to modify my profile (I left the prompt blank to specify my default profile at /home/brian/.bashrc)
Upon inspection, the /home/brian/.bashrc now contains a couple of new 'source' commands to pull in the variables to add to PATH

### To add those new variables to the shell, reload:

    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ exec -l $SHELL
    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ echo $PATH
    /home/brian/google-cloud-sdk/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/brian/Python/pycharm-community-4.0.6/bin:/home/brian/Python/pycharm-community-4.0.6/bin
    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ which gcloud
    /home/brian/google-cloud-sdk/bin/gcloud


### Then, setup gcloud for this machine:
    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ gcloud init


### This asked if I want to login to my google acct.  I answered 'Y', which then opened a browser with my google accounts.  After approving the authorization there, the terminal had the following:

    You are now logged in as: [brianlawney@gmail.com]

    Enter project id you would like to use:  CCCB
    [config set project CCCB] has failed
    You can use [gcloud config] to change more gcloud settings.

    Your current configuration is: [default]

    [app]
    use_appengine_api = True
    [core]
    account = brianlawney@gmail.com
    disable_usage_reporting = True
    [meta]
    active_config = default
    ERROR: (gcloud.init) Failed command: [config set project CCCB] with exit code [1]
    brian@brian-XPS-13-9343 ~/google-cloud-sdk $
    
    
### OK, that was in error because I specified 'CCCB' as the project id.  The actual name is 'dfci-cccb', so let's try again:

    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ gcloud init
    Welcome! This command will take you through the configuration of gcloud.

    Settings from your current configuration [default] are:
    [app]
    use_appengine_api = True
    [core]
    account = brianlawney@gmail.com
    disable_usage_reporting = True
    [meta]
    active_config = default

    Pick configuration to use:
     [1] Re-initialize this configuration [default] with new settings 
     [2] Create a new configuration
    Please enter your numeric choice:  1

    Your current configuration has been set to: [default]

    Pick credentials to use:
     [1] brianlawney@gmail.com
     [2] Login with new credentials
    Please enter your numeric choice:  1

    You are now logged in as: [brianlawney@gmail.com]

    Enter project id you would like to use:  dfci-cccb
    Your current project has been set to: [dfci-cccb].

    Which compute zone would you like to use as project default?
     [1] [asia-east1-c]
     [2] [asia-east1-b]
     [3] [asia-east1-a]
     [4] [europe-west1-d]
     [5] [europe-west1-b]
     [6] [europe-west1-c]
     [7] [us-central1-b]
     [8] [us-central1-a]
     [9] [us-central1-f]
     [10] [us-central1-c]
     [11] [us-east1-b]
     [12] [us-east1-c]
     [13] [us-east1-d]
     [14] Do not set default zone
    Please enter your numeric choice:  11

    Your project default compute region has been set to [us-east1].
    You can change it by running [gcloud config set compute/region NAME].

    Could not retrieve list of repos via [gcloud alpha source repos list]
    Perhaps alpha commands are not enabled or the repos list command failed.

    Generally projects have a repository named [default]. Would you like 
    to try clone it? (Y/n)?  n


    gcloud has now been configured!
    You can use [gcloud config] to change more gcloud settings.

    Your current configuration is: [default]

    [app]
    use_appengine_api = True
    [compute]
    region = us-east1
    zone = us-east1-b
    [core]
    account = brianlawney@gmail.com
    disable_usage_reporting = True
    project = dfci-cccb
    [meta]
    active_config = default
    brian@brian-XPS-13-9343 ~/google-cloud-sdk $


## At this point, we have gcloud setup, but have not done anything on the Python side.  Now install the tools for that.

We follow the tutorial at https://cloud.google.com/compute/docs/tutorials/python-guide#setup

### Before doing this, setup a virtualenv for this, so we don't pollute the global python.  ~/Python/ is where I keep all this personally.  My python (dev version) is located at /usr/local/bin/python2.7

    brian@brian-XPS-13-9343 ~/google-cloud-sdk $ cd ~/Python/
brian@brian-XPS-13-9343 ~/Python $ virtualenv -p /usr/local/bin/python2.7 gcloud_venv
    Running virtualenv with interpreter /usr/local/bin/python2.7
    New python executable in gcloud_venv/bin/python2.7
    Also creating executable in gcloud_venv/bin/python
    Installing setuptools, pip...done.
    brian@brian-XPS-13-9343 ~/Python $ cd gcloud_venv/
brian@brian-XPS-13-9343 ~/Python/gcloud_env $ source bin/activate
    (gcloud_venv)brian@brian-XPS-13-9343 ~/Python/gcloud_venv $ pip install --upgrade google-api-python-client
    
    
### Now we can play around a bit to see how the Python hook to the API works.  

First, to test some of this out, I went to the google developers console, went to Compute -> Compute Engine -> VM instances and started two new f1-micro instances called "instance-1" and "instance-2".  For both:
- I used ubuntu 14.04 LTS on a new 10GB persistent disk (this is 'default')
- I allow http traffic, but not https (also default).
- Everything else is 'default'.  There were some advanced settings that I inspected, but did not change.

### Once those instances were online, try it out:

In [20]:
# for printing the result (which is a json string)
import json 

# the necessary imports specified by the tutorial- the gcloud init step set up all the credentials for this machine
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build

# boilerplate to get things going- 
# 'compute' specifies the service (compute engine) and 'v1' specifies the version of the API 
credentials = GoogleCredentials.get_application_default()
compute = build('compute', 'v1', credentials=credentials)

## Query to see what is currently running:

Note that I did this elsewhere and printed the result, before moving to this notebook.  I copied the result below.  

The query about our current 'instance state' was done with:

<code>
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)
</code>

<code>
{
    "items": [
        {
            "status": "RUNNING", 
            "cpuPlatform": "Intel Haswell", 
            "kind": "compute#instance", 
            "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
            "name": "instance-1", 
            "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
            "tags": {
                "items": [
                    "http-server"
                ], 
                "fingerprint": "FYLDgkTKlA4="
            }, 
            "disks": [
                {
                    "deviceName": "instance-1", 
                    "kind": "compute#attachedDisk", 
                    "autoDelete": true, 
                    "index": 0, 
                    "boot": true, 
                    "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-1", 
                    "interface": "SCSI", 
                    "mode": "READ_WRITE", 
                    "licenses": [
                        "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
                    ], 
                    "type": "PERSISTENT"
                }
            ], 
            "metadata": {
                "kind": "compute#metadata", 
                "fingerprint": "DGgAKiR6VaA="
            }, 
            "scheduling": {
                "automaticRestart": true, 
                "preemptible": false, 
                "onHostMaintenance": "MIGRATE"
            }, 
            "canIpForward": false, 
            "serviceAccounts": [
                {
                    "scopes": [
                        "https://www.googleapis.com/auth/cloud.useraccounts.readonly", 
                        "https://www.googleapis.com/auth/devstorage.read_only", 
                        "https://www.googleapis.com/auth/logging.write", 
                        "https://www.googleapis.com/auth/monitoring.write"
                    ], 
                    "email": "963687796352@project.gserviceaccount.com"
                }
            ], 
            "networkInterfaces": [
                {
                    "accessConfigs": [
                        {
                            "kind": "compute#accessConfig", 
                            "type": "ONE_TO_ONE_NAT", 
                            "name": "External NAT", 
                            "natIP": "104.196.32.48"
                        }
                    ], 
                    "networkIP": "10.240.0.2", 
                    "network": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/networks/default", 
                    "name": "nic0"
                }
            ], 
            "creationTimestamp": "2015-11-11T09:35:45.154-08:00", 
            "id": "147022617172772075", 
            "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/instance-1", 
            "description": ""
        }, 
        {
            "status": "RUNNING", 
            "cpuPlatform": "Intel Haswell", 
            "kind": "compute#instance", 
            "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
            "name": "instance-2", 
            "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
            "tags": {
                "items": [
                    "http-server"
                ], 
                "fingerprint": "FYLDgkTKlA4="
            }, 
            "disks": [
                {
                    "deviceName": "instance-2", 
                    "kind": "compute#attachedDisk", 
                    "autoDelete": true, 
                    "index": 0, 
                    "boot": true, 
                    "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-2", 
                    "interface": "SCSI", 
                    "mode": "READ_WRITE", 
                    "licenses": [
                        "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
                    ], 
                    "type": "PERSISTENT"
                }
            ], 
            "metadata": {
                "kind": "compute#metadata", 
                "fingerprint": "DGgAKiR6VaA="
            }, 
            "scheduling": {
                "automaticRestart": true, 
                "preemptible": false, 
                "onHostMaintenance": "MIGRATE"
            }, 
            "canIpForward": false, 
            "serviceAccounts": [
                {
                    "scopes": [
                        "https://www.googleapis.com/auth/cloud.useraccounts.readonly", 
                        "https://www.googleapis.com/auth/devstorage.read_only", 
                        "https://www.googleapis.com/auth/logging.write", 
                        "https://www.googleapis.com/auth/monitoring.write"
                    ], 
                    "email": "963687796352@project.gserviceaccount.com"
                }
            ], 
            "networkInterfaces": [
                {
                    "accessConfigs": [
                        {
                            "kind": "compute#accessConfig", 
                            "type": "ONE_TO_ONE_NAT", 
                            "name": "External NAT", 
                            "natIP": "104.196.18.196"
                        }
                    ], 
                    "networkIP": "10.240.0.3", 
                    "network": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/networks/default", 
                    "name": "nic0"
                }
            ], 
            "creationTimestamp": "2015-11-11T09:36:01.325-08:00", 
            "id": "9249150384269675832", 
            "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/instance-2", 
            "description": ""
        }
    ], 
    "kind": "compute#instanceList", 
    "id": "projects/dfci-cccb/zones/us-east1-b/instances", 
    "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances"
}
</code>

## Now, shutdown 'instance-2', then run again to see what changed.

The content of the json is not that important, but note that one has status = RUNNING and the other is 'TERMINATED'

In [3]:
# Let's see that this reflects what we just did in the developer console
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "status": "RUNNING", 
            "cpuPlatform": "Intel Haswell", 
            "kind": "compute#instance", 
            "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
            "name": "instance-1", 
            "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
            "tags": {
                "items": [
                    "http-server"
                ], 
                "fingerprint": "FYLDgkTKlA4="
            }, 
            "disks": [
                {
                    "deviceName": "instance-1", 
                    "kind": "compute#attachedDisk", 
                    "autoDelete": true, 
                    "index": 0, 
                    "boot": true, 
                    "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-1", 
                    "interface": "SCSI", 
         

## Deleted both, run again:

In [5]:
# Let's see that this reflects what we just did in the developer console
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)

{
    "kind": "compute#instanceList", 
    "id": "projects/dfci-cccb/zones/us-east1-b/instances", 
    "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances"
}


## OK, so that works fine.  How about managing instances on this end, rather than using the developer console.

From the tutorial page,  I copied the code snippet to launch a new instance (https://github.com/GoogleCloudPlatform/python-docs-samples/blob/master/compute/api/create_instance.py).  Their full example has it launch a startup script, etc.   I don't care about that at the moment, so strip this method down to the bare minimum.

In particular, I remove the metadata section, where they specify commands to launch upon startup in a script.  I also make note of the difference between the 'scopes' variable in the serviceAccounts section:  

Up above, it is:

    "scopes": [
        "https://www.googleapis.com/auth/cloud.useraccounts.readonly", 
        "https://www.googleapis.com/auth/devstorage.read_only", 
        "https://www.googleapis.com/auth/logging.write", 
        "https://www.googleapis.com/auth/monitoring.write"
    ]
           
Below, it is:

    "scopes": [
        "https://www.googleapis.com/auth/devstorage.read_write",
        "https://www.googleapis.com/auth/logging.write"
    ]
    
    
These define which other google services we can use, like cloud storage, access to user accounts, etc.  Eventually we will almost always want read/write access to the cloud object storage, but for the moment let's just leave these as-is.

## Before continuing, we want to get the various public images-- to launch our own ubuntu install, we need to know the full name/url of the ubuntu image.  There are a ton of these...

We are going to use the ID of one of these in the script below, which launches an instance.

In [23]:
ubuntu_images = compute.images().list(project='ubuntu-os-cloud').execute()
print json.dumps(ubuntu_images, indent=4)

{
    "items": [
        {
            "status": "READY", 
            "kind": "compute#image", 
            "sourceType": "RAW", 
            "rawDisk": {
                "containerType": "TAR", 
                "source": ""
            }, 
            "deprecated": {
                "state": "DEPRECATED", 
                "replacement": "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1204-precise-v20150910"
            }, 
            "archiveSizeBytes": "378614228", 
            "diskSizeGb": "10", 
            "licenses": [
                "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1204-precise"
            ], 
            "creationTimestamp": "2014-10-29T09:51:42.018-07:00", 
            "id": "15508054221909398824", 
            "selfLink": "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1204-precise-v20141028", 
            "name": "ubuntu-1204-precise-v20141028"

## Note that I chose ubuntu 14.04, but you have to wade through a bunch of deprecated versions.  The chosen one is:
    https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20150909a
    
### Below, we edit their method to create a simple instance

In [24]:
def create_simple_instance(compute, project, zone, name):
    # the one image resource in the tutorial was:
    # source_disk_image = "projects/debian-cloud/global/images/debian-7-wheezy-v20150320"
    
    # using the json snippet from our manual instance launch above, we can see where the ubuntu 1404 image is located.
    # note that this can be a fullly qualified like in that, but that's a bit unnecessary
    source_disk_image = "projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20150909a"
    # they use a n1 standard in the tutorial:
    # machine_type = "zones/%s/machineTypes/n1-standard-1" % zone

    # here, just launch a f1-micro.
    # also note that here we strip off the "https://www.googleapis.com/compute/v1/projects/dfci-cccb/" prefix
    # It can use either, but not specifying the prefix keeps it 'project agnostic' so this method can be more general
    machine_type = "zones/%s/machineTypes/f1-micro" % zone 

    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],

        # Allow the instance to access cloud storage and logging.
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }]
    }

    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()

## That setup a method to call.  Let's try it out.

In [25]:
i = create_simple_instance(compute, 'dfci-cccb', 'us-east1-b', 'instance-1')

## Print immediately after:

In [27]:
print json.dumps(i, indent=4)

{
    "status": "PENDING", 
    "kind": "compute#operation", 
    "name": "operation-1447344396815-5245a20a3ca98-504b6e87-1d76349f", 
    "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
    "startTime": "2015-11-12T08:06:38.051-08:00", 
    "insertTime": "2015-11-12T08:06:37.683-08:00", 
    "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/instance-1", 
    "operationType": "insert", 
    "progress": 0, 
    "id": "1653955580955430442", 
    "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447344396815-5245a20a3ca98-504b6e87-1d76349f", 
    "user": "brianlawney@gmail.com"
}


## The response is just a json string that acknowledges the request.  Eventually the VM comes online.  

Note that in the github snippet mentioned above, they have a small helper function which alternately polls the new instance and sleeps until it is ready.  We don't need that yet.

### List the current instances to see what it worked:

In [29]:
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "status": "RUNNING", 
            "cpuPlatform": "Intel Haswell", 
            "kind": "compute#instance", 
            "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
            "name": "instance-1", 
            "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
            "tags": {
                "fingerprint": "42WmSpB8rSM="
            }, 
            "disks": [
                {
                    "deviceName": "persistent-disk-0", 
                    "kind": "compute#attachedDisk", 
                    "autoDelete": true, 
                    "index": 0, 
                    "boot": true, 
                    "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-1", 
                    "interface": "SCSI", 
                    "mode": "READ_WRITE", 
                    "licenses": [
      

## Let's stop it:

In [31]:
compute.instances().stop(project='dfci-cccb', zone='us-east1-b', instance='instance-1').execute()

{u'id': u'15180426688476653091',
 u'insertTime': u'2015-11-12T08:21:17.952-08:00',
 u'kind': u'compute#operation',
 u'name': u'operation-1447345277861-5245a55277c8a-cea53d06-e0cded2f',
 u'operationType': u'stop',
 u'progress': 0,
 u'selfLink': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447345277861-5245a55277c8a-cea53d06-e0cded2f',
 u'startTime': u'2015-11-12T08:21:18.318-08:00',
 u'status': u'PENDING',
 u'targetId': u'13491501595124540765',
 u'targetLink': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/instance-1',
 u'user': u'brianlawney@gmail.com',
 u'zone': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b'}

## Let's check the status:

In [32]:
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "status": "TERMINATED", 
            "cpuPlatform": "Unknown CPU Platform", 
            "kind": "compute#instance", 
            "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
            "name": "instance-1", 
            "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
            "tags": {
                "fingerprint": "42WmSpB8rSM="
            }, 
            "disks": [
                {
                    "deviceName": "persistent-disk-0", 
                    "kind": "compute#attachedDisk", 
                    "autoDelete": true, 
                    "index": 0, 
                    "boot": true, 
                    "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-1", 
                    "interface": "SCSI", 
                    "mode": "READ_WRITE", 
                    "licenses"

### Note that its state is 'TERMINATED' (aka stopped).  The disk is persistent, so anything that was on there will remain if/when we start it again.

## Now delete it.

In [33]:
compute.instances().delete(project='dfci-cccb', zone='us-east1-b', instance='instance-1').execute()

{u'id': u'8581248275324901796',
 u'insertTime': u'2015-11-12T08:23:30.589-08:00',
 u'kind': u'compute#operation',
 u'name': u'operation-1447345410508-5245a5d0f84e2-14c872f3-075a98af',
 u'operationType': u'delete',
 u'progress': 0,
 u'selfLink': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447345410508-5245a5d0f84e2-14c872f3-075a98af',
 u'startTime': u'2015-11-12T08:23:30.936-08:00',
 u'status': u'PENDING',
 u'targetId': u'13491501595124540765',
 u'targetLink': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/instance-1',
 u'user': u'brianlawney@gmail.com',
 u'zone': u'https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b'}

## Again, list the instances:

In [36]:
credentials = GoogleCredentials.get_application_default()
compute = build('compute', 'v1', credentials=credentials)
result = compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute()
print json.dumps(result, indent=4)

{
    "kind": "compute#instanceList", 
    "id": "projects/dfci-cccb/zones/us-east1-b/instances", 
    "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances"
}


# Alright, so we can use this python library to do those simple manipulations (create, stop, delete).  How about the cloud storage API?

In [43]:
# re-import just so everything is in one place for clarity
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build

credentials = GoogleCredentials.get_application_default()

# above, we had used build('compute', 'v1', credentials=credentials)
# now, we are accessing the storage API
storage = build('storage', 'v1', credentials=credentials)
result = storage.buckets().list(project='dfci-cccb').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "website": {
                "mainPageSuffix": "index.html"
            }, 
            "kind": "storage#bucket", 
            "name": "www.lung-genomics.org", 
            "timeCreated": "2015-06-04T15:26:22.861Z", 
            "updated": "2015-06-04T16:30:33.971Z", 
            "projectNumber": "963687796352", 
            "metageneration": "4", 
            "location": "US", 
            "owner": {
                "entity": "project-owners-963687796352"
            }, 
            "etag": "CAQ=", 
            "id": "www.lung-genomics.org", 
            "selfLink": "https://www.googleapis.com/storage/v1/b/www.lung-genomics.org", 
            "storageClass": "STANDARD"
        }
    ], 
    "kind": "storage#buckets"
}


### Looking in the console, there was already a single bucket, "www.lung-genomics.org".

## Create a new bucket.

Write a bare minimum function to do this:

In [46]:
def create_simple_bucket(storage, project, name):
    '''
    per the API docs, 
    https://developers.google.com/resources/api-libraries/documentation/storage/v1/python/latest/storage_v1.buckets.html#insert
    
    Need to supply a json string to specify how to configure the bucket
    '''
    
    config = {
        'name': name,
    }

    return storage.buckets().insert(
        project=project,
        body=config).execute()

## Call the method and print the response

In [47]:
result = create_simple_bucket(storage, 'dfci-cccb', 'dfci-demo-bucket')
print json.dumps(result, indent=4)

{
    "kind": "storage#bucket", 
    "name": "dfci-demo-bucket", 
    "timeCreated": "2015-11-12T18:13:01.271Z", 
    "updated": "2015-11-12T18:13:01.271Z", 
    "projectNumber": "963687796352", 
    "metageneration": "1", 
    "location": "US", 
    "owner": {
        "entity": "project-owners-963687796352"
    }, 
    "etag": "CAE=", 
    "id": "dfci-demo-bucket", 
    "selfLink": "https://www.googleapis.com/storage/v1/b/dfci-demo-bucket", 
    "storageClass": "STANDARD"
}


## List the buckets to see that it's there

In [48]:
result = storage.buckets().list(project='dfci-cccb').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "kind": "storage#bucket", 
            "name": "dfci-demo-bucket", 
            "timeCreated": "2015-11-12T18:13:01.271Z", 
            "updated": "2015-11-12T18:13:01.271Z", 
            "projectNumber": "963687796352", 
            "metageneration": "1", 
            "location": "US", 
            "owner": {
                "entity": "project-owners-963687796352"
            }, 
            "etag": "CAE=", 
            "id": "dfci-demo-bucket", 
            "selfLink": "https://www.googleapis.com/storage/v1/b/dfci-demo-bucket", 
            "storageClass": "STANDARD"
        }, 
        {
            "website": {
                "mainPageSuffix": "index.html"
            }, 
            "kind": "storage#bucket", 
            "name": "www.lung-genomics.org", 
            "timeCreated": "2015-06-04T15:26:22.861Z", 
            "updated": "2015-06-04T16:30:33.971Z", 
            "projectNumber": "963687796352", 
            "metageneration": "4", 


## Let's query the bucket.

In [52]:
result = storage.objects().list(bucket='dfci-demo-bucket').execute()
print result

{u'kind': u'storage#objects'}


## As we know, it's empty.  Try to upload a picture.  We will use the picture below  

<img src="CCCB_Logo.png"></img>

In [63]:
# re-import just so everything is in one place for clarity
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build

credentials = GoogleCredentials.get_application_default()
storage = build('storage', 'v1', credentials=credentials)

# the default io library has various classes for creating bitstreams,etc.
import io

# import this class which constructs the proper request to the google storage API
from googleapiclient.http import MediaIoBaseUpload

# create a file 'handle' and construct the object which will be uploaded
fh = io.FileIO('CCCB_Logo.png')
media = MediaIoBaseUpload(fh, mimetype='image/png')

# send it!
result = storage.objects().insert(bucket='dfci-demo-bucket', name="cccb-logo", media_body=media).execute()


### Print the response from the insertion:

In [64]:
print json.dumps(result, indent=4)

{
    "kind": "storage#object", 
    "contentType": "image/png", 
    "name": "cccb-logo", 
    "timeCreated": "2015-11-12T19:18:47.562Z", 
    "generation": "1447355927564000", 
    "md5Hash": "AXo9WSNZYvGiH9N0X0RPEQ==", 
    "bucket": "dfci-demo-bucket", 
    "updated": "2015-11-12T19:18:47.562Z", 
    "owner": {
        "entityId": "00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f", 
        "entity": "user-00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f"
    }, 
    "crc32c": "4q3RtA==", 
    "metageneration": "1", 
    "storageClass": "STANDARD", 
    "mediaLink": "https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447355927564000&alt=media", 
    "etag": "COCdtP/Mi8kCEAE=", 
    "id": "dfci-demo-bucket/cccb-logo/1447355927564000", 
    "selfLink": "https://www.googleapis.com/storage/v1/b/dfci-demo-bucket/o/cccb-logo", 
    "size": "20536"
}


## Query the bucket to see that the new image is there

In [66]:
result = storage.objects().list(bucket='dfci-demo-bucket').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "kind": "storage#object", 
            "contentType": "image/png", 
            "name": "cccb-logo", 
            "timeCreated": "2015-11-12T19:18:47.562Z", 
            "generation": "1447355927564000", 
            "md5Hash": "AXo9WSNZYvGiH9N0X0RPEQ==", 
            "bucket": "dfci-demo-bucket", 
            "updated": "2015-11-12T19:18:47.562Z", 
            "owner": {
                "entityId": "00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f", 
                "entity": "user-00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f"
            }, 
            "crc32c": "4q3RtA==", 
            "metageneration": "1", 
            "storageClass": "STANDARD", 
            "mediaLink": "https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447355927564000&alt=media", 
            "etag": "COCdtP/Mi8kCEAE=", 
            "id": "dfci-demo-bucket/cccb-logo/1447355927564000", 
           

### Note that if you try to access it via the link below (which is copied from the json above), it says 'login required'.

https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447355426361000&alt=media
(Note: this link is likely not current/available in this notebook as things change while I test)

In the developer console, make this PNG public (via the checkbox) and query again:

In [67]:
result = storage.objects().list(bucket='dfci-demo-bucket').execute()
print json.dumps(result, indent=4)

{
    "items": [
        {
            "kind": "storage#object", 
            "contentType": "image/png", 
            "name": "cccb-logo", 
            "timeCreated": "2015-11-12T19:18:47.562Z", 
            "generation": "1447355927564000", 
            "md5Hash": "AXo9WSNZYvGiH9N0X0RPEQ==", 
            "bucket": "dfci-demo-bucket", 
            "updated": "2015-11-12T19:20:01.023Z", 
            "owner": {
                "entityId": "00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f", 
                "entity": "user-00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f"
            }, 
            "crc32c": "4q3RtA==", 
            "metageneration": "2", 
            "storageClass": "STANDARD", 
            "mediaLink": "https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447355927564000&alt=media", 
            "etag": "COCdtP/Mi8kCEAI=", 
            "id": "dfci-demo-bucket/cccb-logo/1447355927564000", 
           

### Nothing apparently changed in the json response, but it's now accessible via the browser (publically).

### There is also a flag you can use to make the image public by default.  Let's try that.  

First, I deleted the png in the developer console.  Then try upload again.  This time, we use a 'predefinedAcl' argument which accepts a string denoting one of the several standard/common access patterns.  You can define custom access to various groups/individuals, etc. but here we just use the public read option ('publicRead').  This is effectively a chmod 774 operation, as far as I can tell. 

In [69]:
result = storage.objects().list(bucket='dfci-demo-bucket').execute()
print json.dumps(result, indent=4)

print '*'*50

fh = io.FileIO('CCCB_Logo.png')
media = MediaIoBaseUpload(fh, mimetype='image/png')

result = storage.objects().insert(bucket='dfci-demo-bucket', name="cccb-logo", media_body=media, predefinedAcl='publicRead').execute()

print '*'*50

result = storage.objects().list(bucket='dfci-demo-bucket').execute()
print json.dumps(result, indent=4)



{
    "kind": "storage#objects"
}
**************************************************
**************************************************
{
    "items": [
        {
            "kind": "storage#object", 
            "contentType": "image/png", 
            "name": "cccb-logo", 
            "timeCreated": "2015-11-12T19:34:52.741Z", 
            "generation": "1447356892742000", 
            "md5Hash": "AXo9WSNZYvGiH9N0X0RPEQ==", 
            "bucket": "dfci-demo-bucket", 
            "updated": "2015-11-12T19:34:52.741Z", 
            "owner": {
                "entityId": "00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f", 
                "entity": "user-00b4903a978eff972f57dd3395dee21b6215fb08ec142237294030035ec9af8f"
            }, 
            "crc32c": "4q3RtA==", 
            "metageneration": "1", 
            "storageClass": "STANDARD", 
            "mediaLink": "https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=144735689274

## Check that it works:

    <img src="https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447356892742000&alt=media">

<img src="https://www.googleapis.com/download/storage/v1/b/dfci-demo-bucket/o/cccb-logo?generation=1447356892742000&alt=media">

### Note that "re-compiling" the above cell may break it-- when created, the image was actually referencing the google storage bucket.  As the bucket might change, etc. the reference could be lost.  In any case, it worked when created. 

# OK, so far we have played with instances, and a little with the storage bucket.

### Let's do a demo orchestration where we have a main VM which launches a couple of others that do some 'work' and drop their results into the storage bucket.

To do this, we will have a main VM, to which we will login via ssh.  In practice, the analysis portal will be a web app which will construct the appropriate work queue; here, we just will have a simple script that will abstract that process.  From there, we will create two worker instances which will process their respective 'jobs', which will just be some simple operation.  

To mock how an alignment process might work, we put some input files into a bucket.  Each worker machine will be told which files to process.  They will retrieve the input file(s), do their work, and put output files in the same storage bucket.  Then they will shut down.

For now, these worker VMs will be just default micro instances.  Once we're up and running, these will be pre-configured VM images which will have all the necessary software installed.

## Create a 'startup' script which will do some task.  We will pass parameters for the script in as key-value pairs via the metadata field when writing the instance creation request.  As each instance is launched, it will pull that script, pass in the appropriate parameters and do its task.

I created two scripts, printed below.  Both of these are stored in the dfci-demo-bucket.  

The first is a shell script which is automatically executed when the new instances boot.  This shell script queries for parameters that were passed with the request to launch the instance (e.g. input files to work on, output files, which storage bucket to use).  It pulls in some resources from the storage bucket, does some trivial work, writes the results back to the storage bucket, and then kills the current instance.

The second script mocks some 'data processing' that could occur.  In this case, it just takes an input file at a specified location, repeats the file's contents a number of times, and writes the output to a file which is then copied into the storage bucket.

In [1]:
! cat ss_script.sh

print '#'*50

! cat dummy_process.py

#!/bin/bash

PROJECT_ID=$(curl http://metadata/computeMetadata/v1/instance/attributes/project_id -H "X-Google-Metadata-Request: True")
INPUT_FILE=$(curl http://metadata/computeMetadata/v1/instance/attributes/input_filename -H "X-Google-Metadata-Request: True")
N=$(curl http://metadata/computeMetadata/v1/instance/attributes/repeats -H "X-Google-Metadata-Request: True")
OUTPUT_FILE=$(curl http://metadata/computeMetadata/v1/instance/attributes/output_filename -H "X-Google-Metadata-Request: True")
BUCKET=$(curl http://metadata/computeMetadata/v1/instance/attributes/bucket -H "X-Google-Metadata-Request: True")
INSTANCE_NAME=$(curl http://metadata/computeMetadata/v1/instance/attributes/instance_name -H "X-Google-Metadata-Request: True")
PY_SCRIPT=$(curl http://metadata/computeMetadata/v1/instance/attributes/py_script -H "X-Google-Metadata-Request: True")

# copy the python script that mocks a 'real' process
gsutil cp gs://$BUCKET/$PY_SCRIPT .

# copy the input 'data file'
gsutil cp gs://$BUC

## To give the startup shell script the proper arguments, we had to pass them in via the metadata field in the VM creation request.  Below, I copy the method.

The script is a quick-and-dirty way to launch the instance with the required parameters.

<code>
def create_simple_instance(compute, project, zone, name, **kwargs):

    bucket = kwargs['bucket']
    cccb_project_id = kwargs['cccb_project_id']
    input_file = kwargs['input_file']
    repeats = kwargs['repeats']
    output_file = kwargs['output_file']
    processing_script = kwargs['processing_script']

    startup_script_url = 'gs://%s/ss_script.sh' % bucket

    source_disk_image = "projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20150909a"

    machine_type = "zones/%s/machineTypes/f1-micro" % zone 

    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],

        # Allow the instance to access cloud storage and logging.
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/compute',
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }],
 'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script-url',
                'value': startup_script_url
            }, {
                'key': 'input_filename',
                'value': input_file
            }, {
		'key' : 'project_id',
		'value': cccb_project_id
	    }, {
                'key': 'repeats',
                'value': repeats
            }, {
                'key': 'bucket',
                'value': bucket
            }, {
                'key':'output_filename', 
                'value': output_file
            }, {
                'key':'instance_name', 
                'value':name
            }, {
                'key':'py_script', 
                'value': processing_script
            }
          ]
        }
    }

    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()


\# the necessary imports specified by the tutorial- the gcloud init step set up all the credentials for this machine
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build
import sys

if __name__ == '__main__':
	credentials = GoogleCredentials.get_application_default()
	compute = build('compute', 'v1', credentials=credentials)

	processing_script = 'dummy_process.py'

	instance_name = sys.argv[1]
	bucket = sys.argv[2]
	input_file = sys.argv[3]
	output_file = sys.argv[4]
	cccb_project_id = sys.argv[5]
	repeats = int(sys.argv[6])

	kwargs = {}
	kwargs['bucket'] = bucket
	kwargs['cccb_project_id'] =  cccb_project_id 
	kwargs['input_file'] = input_file
	kwargs['repeats'] = repeats
	kwargs['output_file'] = output_file
	kwargs['processing_script'] = processing_script

	i = create_simple_instance(compute, 'dfci-cccb', 'us-east1-b', instance_name, **kwargs)
</code>

## To mock multiple instances/jobs, I call that script (create.py) with various input arguments.  Here, the name of the bucket, the 'cccb project id's, and the filenames are all known in advance.  In practice, this would all be something we would have to determine.  However, the functionality is the same, and it's an easy proof-of-concept. 
<code>
#!/bin/bash
/home/brianlawney/venv/bin/python create.py instance-a dfci-demo-bucket fileA.txt fileA.processed.txt AB-12345-000 3
sleep 10
/home/brianlawney/venv/bin/python create.py instance-b dfci-demo-bucket fileB.txt fileB.processed.txt CD-12345-111 5
</code>

### Sure enough, the appropriate files show up in the storage bucket.  In the compute history, we can see the launch and subsequent deletion of those worker VMs:
<img src="vm_history.png"></img>

# Eventually, we want to have pre-configured machines with all the necessary software.  

To do a quick test, I will start a vanilla ubuntu VM, then load in some python packages.  Then, stop the instance, and attempt to create a new image based on that customization.  

The instance was started via the developer console, and I allowed read/write to compute and storage.

I then used their browser-based terminal to issue the following commands:
<code>
brianlawney@custom-instance-test:~$ sudo apt-get update
brianlawney@custom-instance-test:~$ sudo apt-get install python2.7-dev
brianlawney@custom-instance-test:~$ sudo apt-get install python-virtualenv
brianlawney@custom-instance-test:~$ virtualenv venv
brianlawney@custom-instance-test:~$ cd venv
brianlawney@custom-instance-test:~/venv$ source bin/activate
(venv)brianlawney@custom-instance-test:~/venv$ pip install numpy
...
Successfully installed numpy
Cleaning up...
(venv)brianlawney@custom-instance-test:~/venv$ bin/python
Python 2.7.6 (default, Jun 22 2015, 17:58:13) 
[GCC 4.8.2] on linux2
Type "help", "copyright", "credits" or "license" for more information.
>>> import numpy as np
>>> 
</code>

This confirms that we have numpy installed on the system.  Now let's attempt to make a 'custom image' based on this.

In [5]:
import json
from oauth2client.client import GoogleCredentials 
from googleapiclient.discovery import build
credentials = GoogleCredentials.get_application_default()
compute = build('compute', 'v1', credentials=credentials)

# check the current instances:
print json.dumps(compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "RUNNING", 
      "cpuPlatform": "Intel Haswell", 
      "kind": "compute#instance", 
      "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
      "name": "custom-instance-test", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "tags": {
        "items": [
          "http-server"
        ], 
        "fingerprint": "FYLDgkTKlA4="
      }, 
      "disks": [
        {
          "deviceName": "custom-instance-test", 
          "kind": "compute#attachedDisk", 
          "autoDelete": true, 
          "index": 0, 
          "boot": true, 
          "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance-test", 
          "interface": "SCSI", 
          "mode": "READ_WRITE", 
          "licenses": [
            "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ub

## To create an image, need to stop it:

In [6]:
print json.dumps(compute.instances().stop(project='dfci-cccb', zone='us-east1-b', instance='custom-instance-test').execute(), indent=2)

{
  "status": "PENDING", 
  "kind": "compute#operation", 
  "name": "operation-1447435726307-5246f644d3eb9-07376c63-e3a7d48d", 
  "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
  "startTime": "2015-11-13T09:28:46.749-08:00", 
  "insertTime": "2015-11-13T09:28:46.382-08:00", 
  "targetId": "15186138848262653722", 
  "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/custom-instance-test", 
  "operationType": "stop", 
  "progress": 0, 
  "id": "2483321319976814698", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447435726307-5246f644d3eb9-07376c63-e3a7d48d", 
  "user": "brianlawney@gmail.com"
}


## Verify we stopped it:

In [12]:
print json.dumps(compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "TERMINATED", 
      "cpuPlatform": "Unknown CPU Platform", 
      "kind": "compute#instance", 
      "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
      "name": "custom-instance-test", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "tags": {
        "items": [
          "http-server"
        ], 
        "fingerprint": "FYLDgkTKlA4="
      }, 
      "disks": [
        {
          "deviceName": "custom-instance-test", 
          "kind": "compute#attachedDisk", 
          "autoDelete": true, 
          "index": 0, 
          "boot": true, 
          "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance-test", 
          "interface": "SCSI", 
          "mode": "READ_WRITE", 
          "licenses": [
            "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/l

## Check out the available disks in this region:

In [9]:
print json.dumps(compute.disks().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "READY", 
      "sizeGb": "10", 
      "kind": "compute#disk", 
      "name": "custom-instance-test", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "type": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/diskTypes/pd-standard", 
      "sourceImageId": "7281760888248275729", 
      "lastAttachTimestamp": "2015-11-13T08:39:30.889-08:00", 
      "sourceImage": "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20150909a", 
      "licenses": [
        "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
      ], 
      "creationTimestamp": "2015-11-13T08:39:29.983-08:00", 
      "id": "1231914055785753774", 
      "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance-test", 
      "users": [
        "https://www.googleapis.com/c

## As it stands now, I cannot create an image from this disk because it is attached to the custom-instance-test instance (even though the instance is stopped).  

The disk will delete when the instance deletes, so first I need to set the auto delete to false so the disk persists upon deletion of the instance.  THEN we can create the image.  

From the API reference:

<code>
setDiskAutoDelete(project=*, zone=*, instance=*, autoDelete=*, deviceName=*)

Sets the auto-delete flag for a disk attached to an instance.

Args:
  project: string, Project ID for this request. (required)
  zone: string, The name of the zone for this request. (required)
  instance: string, The instance name. (required)
  autoDelete: boolean, Whether to auto-delete the disk when the instance is deleted. (required)
  deviceName: string, The device name of the disk to modify. (required)
</code>

In [14]:
print json.dumps(compute.instances().setDiskAutoDelete(project='dfci-cccb', zone='us-east1-b', instance='custom-instance-test', autoDelete=False, deviceName='custom-instance-test').execute(), indent=2)

{
  "status": "PENDING", 
  "kind": "compute#operation", 
  "name": "operation-1447437865628-5246fe3d0b160-0608e143-a651ffbb", 
  "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
  "startTime": "2015-11-13T10:04:26.133-08:00", 
  "insertTime": "2015-11-13T10:04:25.799-08:00", 
  "targetId": "15186138848262653722", 
  "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/custom-instance-test", 
  "operationType": "setDiskAutoDelete", 
  "progress": 0, 
  "id": "14329339569208394084", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447437865628-5246fe3d0b160-0608e143-a651ffbb", 
  "user": "brianlawney@gmail.com"
}


## List the instances, and note that in the 'disks' section, the autoDelete is false.  Then, delete the 'custom-instance-test' instance

In [16]:
print json.dumps(compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "TERMINATED", 
      "cpuPlatform": "Unknown CPU Platform", 
      "kind": "compute#instance", 
      "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
      "name": "custom-instance-test", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "tags": {
        "items": [
          "http-server"
        ], 
        "fingerprint": "FYLDgkTKlA4="
      }, 
      "disks": [
        {
          "deviceName": "custom-instance-test", 
          "kind": "compute#attachedDisk", 
          "autoDelete": false, 
          "index": 0, 
          "boot": true, 
          "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance-test", 
          "interface": "SCSI", 
          "mode": "READ_WRITE", 
          "licenses": [
            "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/

In [17]:
print json.dumps(compute.instances().delete(project='dfci-cccb', zone='us-east1-b', instance='custom-instance-test').execute(), indent=2)

{
  "status": "PENDING", 
  "kind": "compute#operation", 
  "name": "operation-1447438012467-5246fec914739-ac97f54c-2aa3215b", 
  "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
  "startTime": "2015-11-13T10:06:52.903-08:00", 
  "insertTime": "2015-11-13T10:06:52.582-08:00", 
  "targetId": "15186138848262653722", 
  "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/custom-instance-test", 
  "operationType": "delete", 
  "progress": 0, 
  "id": "7777197302805342871", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447438012467-5246fec914739-ac97f54c-2aa3215b", 
  "user": "brianlawney@gmail.com"
}


## Confirm the deletion:

In [18]:
print json.dumps(compute.instances().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "TERMINATED", 
      "cpuPlatform": "Unknown CPU Platform", 
      "kind": "compute#instance", 
      "machineType": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/machineTypes/f1-micro", 
      "name": "instance-2", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "tags": {
        "items": [
          "http-server"
        ], 
        "fingerprint": "FYLDgkTKlA4="
      }, 
      "disks": [
        {
          "deviceName": "instance-2", 
          "kind": "compute#attachedDisk", 
          "autoDelete": true, 
          "index": 0, 
          "boot": true, 
          "source": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/instance-2", 
          "interface": "SCSI", 
          "mode": "READ_WRITE", 
          "licenses": [
            "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
  

## Ensure the disk is still there:

In [19]:
print json.dumps(compute.disks().list(project='dfci-cccb', zone='us-east1-b').execute(), indent=2)

{
  "items": [
    {
      "status": "READY", 
      "sizeGb": "10", 
      "kind": "compute#disk", 
      "lastDetachTimestamp": "2015-11-13T10:06:55.859-08:00", 
      "name": "custom-instance-test", 
      "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
      "type": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/diskTypes/pd-standard", 
      "sourceImageId": "7281760888248275729", 
      "lastAttachTimestamp": "2015-11-13T08:39:30.889-08:00", 
      "sourceImage": "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20150909a", 
      "licenses": [
        "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
      ], 
      "creationTimestamp": "2015-11-13T08:39:29.983-08:00", 
      "id": "1231914055785753774", 
      "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance

## Now try and make the image

In [20]:
config = {
    'name':'my-custom-image',
    'sourceDisk':'zones/us-east1-b/disks/custom-instance-test'
}

print json.dumps(compute.images().insert(project='dfci-cccb', body=config).execute(), indent=2)

{
  "status": "PENDING", 
  "kind": "compute#operation", 
  "name": "operation-1447438122035-5246ff3192739-cc4198d5-d80f992b", 
  "insertTime": "2015-11-13T10:08:42.240-08:00", 
  "targetId": "8493252647081187781", 
  "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/images/my-custom-image", 
  "operationType": "insert", 
  "progress": 0, 
  "id": "5397324006899292613", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/operations/operation-1447438122035-5246ff3192739-cc4198d5-d80f992b", 
  "user": "brianlawney@gmail.com"
}


## Confirm we made it into the 'list':

In [21]:
print json.dumps(compute.images().list(project='dfci-cccb', filter='name eq my-custom-image').execute(), indent=2)

{
  "items": [
    {
      "status": "READY", 
      "kind": "compute#image", 
      "description": "", 
      "sourceDisk": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/disks/custom-instance-test", 
      "archiveSizeBytes": "700191246", 
      "sourceDiskId": "1231914055785753774", 
      "diskSizeGb": "10", 
      "licenses": [
        "https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/licenses/ubuntu-1404-trusty"
      ], 
      "sourceType": "RAW", 
      "creationTimestamp": "2015-11-13T10:08:42.168-08:00", 
      "id": "8493252647081187781", 
      "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/images/my-custom-image", 
      "name": "my-custom-image"
    }
  ], 
  "kind": "compute#imageList", 
  "id": "projects/dfci-cccb/global/images", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/global/images"
}


## Launch and instance based on this.

In [33]:
def launch_custom_instance(compute, project, zone, name, **kwargs):

    bucket = kwargs['bucket']
    size = kwargs['size']
    output_file = kwargs['output_file']
    processing_script = kwargs['processing_script']
    ss_script = kwargs['ss_script']

    startup_script_url = 'gs://%s/%s' % (bucket, ss_script)
    print startup_script_url

    source_disk_image = "projects/dfci-cccb/global/images/my-custom-image"

    machine_type = "zones/%s/machineTypes/f1-micro" % zone 

    config = {
        'name': name,
        'machineType': machine_type,

        # Specify the boot disk and the image to use as a source.
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],

        # Specify a network interface with NAT to access the public
        # internet.
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],

        # Allow the instance to access cloud storage and logging.
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/compute',
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }],
 'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script-url',
                'value': startup_script_url
            }, {
                'key': 'size',
                'value': size
            }, {
                'key': 'bucket',
                'value': bucket
            }, {
                'key':'output_filename', 
                'value': output_file
            }, {
                'key':'instance_name', 
                'value':name
            }, {
                'key':'py_script', 
                'value': processing_script
            }
          ]
        }
    }

    return compute.instances().insert(
        project=project,
        zone=zone,
        body=config).execute()


kwargs = {}
kwargs['bucket'] = 'dfci-demo-bucket'
kwargs['size'] = '5'
kwargs['output_file'] = 'numpy_result.tsv'
kwargs['processing_script'] = 'numpy_test.py'
kwargs['ss_script'] = 'custom_image_ss_script.sh'

print json.dumps(launch_custom_instance(compute, 'dfci-cccb', 'us-east1-b', 'numpy-test-instance', **kwargs), indent=2)



gs://dfci-demo-bucket/custom_image_ss_script.sh
{
  "status": "PENDING", 
  "kind": "compute#operation", 
  "name": "operation-1447443772855-5247143e9d2d8-bb6849d0-06dc7378", 
  "zone": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b", 
  "startTime": "2015-11-13T11:42:53.849-08:00", 
  "insertTime": "2015-11-13T11:42:53.551-08:00", 
  "targetLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/instances/numpy-test-instance", 
  "operationType": "insert", 
  "progress": 0, 
  "id": "12779010648553689100", 
  "selfLink": "https://www.googleapis.com/compute/v1/projects/dfci-cccb/zones/us-east1-b/operations/operation-1447443772855-5247143e9d2d8-bb6849d0-06dc7378", 
  "user": "brianlawney@gmail.com"
}


## That worked-- it found the existing image (which had python's numpy library), successfully used it, dumped the results in the bucket, and shut down automatically.